In [18]:
import requests as r
from bs4 import BeautifulSoup as bs
import pandas as pd
from time import sleep


In [2]:
url = 'https://ikman.lk/en/ads/colombo/houses'

In [3]:
def fetch_page(url, page):
    res = r.get(url, params={'page': page, 'by_paying_member': 0, 'sort': 'date', 'order': 'desc'})
    return res.content;

In [4]:
def extract_house_url(content):
    soup = bs(content)
    url_container = soup.findAll('a', {'class': 'card-link--3ssYv'})
    urls = []
    for index in range(len(url_container)):
       try:
            urlDom = url_container[index]['href']
       except IndexError:
           continue
       urls.append('https://ikman.lk' + urlDom)
    return urls

In [5]:
con = fetch_page(url, 1)
# print(con)
urls = extract_house_url(con)
print(len(set(urls)))


28


In [6]:
page_max =223;
urls = []
error_pgs = []
for pg in range(1,224):
    try: 
        con = fetch_page(url, pg)
        urls = urls + extract_house_url(con)
    except:
        error_pgs.append(pg)
        print("error: " + str(pg))
    print(pg)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223


In [8]:
urls = list(set(urls))

In [9]:
urls[0]

'https://ikman.lk/en/ad/3-storeyed-house-for-sale-in-kolonnawa-for-sale-colombo'

In [10]:
import pickle

In [12]:
with open('urls_v2', 'wb') as file:
    pickle.dump(urls, file)

In [13]:
def extract_house(url):
    title= ''
    location= ''
    posted_date = ''
    description = ''
    beds = 0
    baths = 0
    price = 0
    house_size = '0.0 sqft'
    land_size = '0.0 perches'
    address = ''
    
    res = r.get(url)
    con = res.content;
    soup = bs(con)
    #contains title
    top_elem = soup.findAll('div', { 'class': 'item-top'})
    
    if (len(top_elem) > 0):
        top_elem = top_elem[0]
        title_elem = top_elem.find_all('h1')
        if (len(title_elem) > 0):
            title = title_elem[0].text
    #contains posted date and location
    intr_elem = soup.findAll('p', {'class': 'item-intro'})
    if (len(intr_elem) >0):
            intr_elem = intr_elem[0]
            posted_date_elem = intr_elem.find_all('span', {'class': 'date'})
            if (len(posted_date_elem) > 0):
                posted_date_elem = posted_date_elem[0]
                posted_date = posted_date_elem.text
            location_elem = intr_elem.find_all('span', {'class': 'location'})
            if (len(location_elem) > 0):
                location_elem = location_elem[0]
                location = location_elem.text
            price_elem = soup.findAll('span', {'class': 'amount'})
            if (len(price_elem) > 0):
                price_elem = price_elem[0]
                price = price_elem.text
            desc_elem = soup.findAll('div', {'class': 'item-description'})
            if (len(desc_elem) > 0):
                desc_elem = desc_elem[0]
                desc_elem = desc_elem.find_all('p')
                if (len(desc_elem) > 0):
                    desc_elem = desc_elem[0]
                    description = desc_elem.text
            prop_elem = soup.findAll('div', {'class': 'item-properties'})
            if (len(prop_elem) > 0):
                prop_elem = prop_elem[0]
                key_elem = prop_elem.find_all('dt')
                val_elem = prop_elem.find_all('dd')
                if(len(key_elem) > 0 and len(val_elem) > 0):
                    for index, key in enumerate(key_elem):
                        if (key.text == 'Address:'):
                            address = val_elem[index].text
                        if (key.text == 'Beds:'):
                            beds = val_elem[index].text
                        if (key.text == 'Baths:'):
                            baths = val_elem[index].text
                        if (key.text == 'House size:'):
                            house_size = val_elem[index].text
                        if (key.text == 'Land size:'):
                            land_size = val_elem[index].text   
    #return list in order title, posted date, location, price , description, address, beds, baths, house size, land size
    row = []
    row.append(title)
    row.append(posted_date)
    row.append(location)
    row.append(price)
    row.append(description)
    row.append(address)
    row.append(beds)
    row.append(baths)
    row.append(house_size)
    row.append(land_size)
    return row;
                    
                    
    
    

In [14]:
print(extract_house('https://ikman.lk/en/ad/architecturally-design-two-storied-house-athurugiriya-for-sale-colombo-4'))

['This item has been sold!', '', '', 0, '', '', 0, 0, '0.0 sqft', '0.0 perches']


In [15]:
df_data = []
missing_urls = []

In [16]:
for index, url in enumerate(urls):
    print(f'{index} of {len(urls)}')
    try:
        print()
        df_data.append(extract_house(url))
    except :
        missing_urls.append(url)
      

0 of 5147

1 of 5147

2 of 5147

3 of 5147

4 of 5147

5 of 5147

6 of 5147

7 of 5147

8 of 5147

9 of 5147

10 of 5147

11 of 5147

12 of 5147

13 of 5147

14 of 5147

15 of 5147

16 of 5147

17 of 5147

18 of 5147

19 of 5147

20 of 5147

21 of 5147

22 of 5147

23 of 5147

24 of 5147

25 of 5147

26 of 5147

27 of 5147

28 of 5147

29 of 5147

30 of 5147

31 of 5147

32 of 5147

33 of 5147

34 of 5147

35 of 5147

36 of 5147

37 of 5147

38 of 5147

39 of 5147

40 of 5147

41 of 5147

42 of 5147

43 of 5147

44 of 5147

45 of 5147

46 of 5147

47 of 5147

48 of 5147

49 of 5147

50 of 5147

51 of 5147

52 of 5147

53 of 5147

54 of 5147

55 of 5147

56 of 5147

57 of 5147

58 of 5147

59 of 5147

60 of 5147

61 of 5147

62 of 5147

63 of 5147

64 of 5147

65 of 5147

66 of 5147

67 of 5147

68 of 5147

69 of 5147

70 of 5147

71 of 5147

72 of 5147

73 of 5147

74 of 5147

75 of 5147

76 of 5147

77 of 5147

78 of 5147

79 of 5147

80 of 5147

81 of 5147

82 of 5147

83 of 5147

84

In [ ]:
len(missing_urls)

In [ ]:
len(df_data)

In [18]:
try:
    df = pd.DataFrame(df_data)
    df.to_csv( r"/" + 'house-data-set' + '.csv')
    print("success csv" + " " + dir)
except TypeError:
        print('Exception in writing csv file')

Exception in writing csv file


In [20]:
df = pd.DataFrame(df_data)
df.to_csv( r"./" + 'house-data-set_v1' + '.csv')

Merge 2 data sets

In [21]:
df_old = pd.read_csv('./data_set_column_adjusted.csv')

In [22]:
df_old.head()


,Unnamed: 0,0,1,2,3,4,5,6,7,8,9
0,0,New House for Sale in Piliyandala,10 Aug 7:49 pm,"Piliyandala, Colombo",12500000,පිලියන්දල මඩපාත devid pieris ආයතනය අසල ඉදිකරමි...,NaN,4,2,"1,750.0 sqft",6.5 perches
1,1,Brand New Luxury Two Storey House Piliyanadala,2 Jul 8:33 am,"Kottawa, Colombo",29500000,Brand new luxury two storey house for sale in ...,kottawa 255 piliyanadala Road,4,4,"3,510.0 sqft",9.0 perches
2,2,Brand New 3 Storied House for Sale in Malabe,16 Aug 8:14 pm,"Malabe, Colombo",19000000,Brand New 3 storied house for sale at malabe ....,malabe,4,3,"2,850.0 sqft",7.5 perches
3,3,House for sale - Kottawa,16 Aug 8:32 pm,"Kottawa, Colombo",35000000,House for sale in Kottawa===================,NaN,7,4,"1,700.0 sqft",15.0 perches
4,4,Brand New Luxury House for sale in Boralesgamuwa,8 Aug 7:51 pm,"Boralesgamuwa, Colombo",29500000,"Brand New, 3 Storied 5 Bed Room House for Sale...",NaN,5,4,"3,500.0 sqft",8.0 perches


In [23]:
df_new = pd.read_csv('./house-data-set-modified.csv')

In [24]:
df_new.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9
0,0,House for Sale Mount Lavinia,1 Feb 8:16 pm,"Mount Lavinia, Colombo",22000000,Fully tiled house 1 vehicle parking inside the...,mountlavina uludhagoda road,3,2,"2,500.0 sqft",6.5 perches
1,1,Brand New Luxury Type Architect Designed House...,1 Feb 7:11 pm,"Malabe, Colombo",24000000,"2 Storey House for Sale in Amaragoda rd, sand...","Malabe, sandoris jayantha mw",4,4,"2,600.0 sqft",8.0 perches
2,2,House for Sale in Piliyandala,30 Jan 3:24 pm,"Piliyandala, Colombo",16900000,"Off Madapatha Road Hillcrest Estate, gated com...",NaN,3,2,"1,000.0 sqft",12.5 perches
3,3,House for Sale in Kawdana Dehiwala,1 Feb 7:07 pm,"Dehiwala, Colombo",30000000,"DetailsProperty Type:Price: Rs. 30,000,000 ($ ...",Palliyadora Road Kawdana,3,3,"2,000.0 sqft",10.0 perches
4,4,Brand New House For Sale In Malabe,4 Jan 7:07 pm,"Malabe, Colombo",10800000,Brand New Single Storied House for Sale @ Mala...,"Robat Gunawardena Mawatha,Malabe",3,3,"1,250.0 sqft",6.5 perches


In [39]:
df_merged = pd.concat([df_old, df_new], ignore_index=True)

In [40]:
df_merged.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9
0,0,New House for Sale in Piliyandala,10 Aug 7:49 pm,"Piliyandala, Colombo",12500000,පිලියන්දල මඩපාත devid pieris ආයතනය අසල ඉදිකරමි...,NaN,4,2,"1,750.0 sqft",6.5 perches
1,1,Brand New Luxury Two Storey House Piliyanadala,2 Jul 8:33 am,"Kottawa, Colombo",29500000,Brand new luxury two storey house for sale in ...,kottawa 255 piliyanadala Road,4,4,"3,510.0 sqft",9.0 perches
2,2,Brand New 3 Storied House for Sale in Malabe,16 Aug 8:14 pm,"Malabe, Colombo",19000000,Brand New 3 storied house for sale at malabe ....,malabe,4,3,"2,850.0 sqft",7.5 perches
3,3,House for sale - Kottawa,16 Aug 8:32 pm,"Kottawa, Colombo",35000000,House for sale in Kottawa===================,NaN,7,4,"1,700.0 sqft",15.0 perches
4,4,Brand New Luxury House for sale in Boralesgamuwa,8 Aug 7:51 pm,"Boralesgamuwa, Colombo",29500000,"Brand New, 3 Storied 5 Bed Room House for Sale...",NaN,5,4,"3,500.0 sqft",8.0 perches


In [41]:
df_merged.shape

(7612, 11)

In [42]:
cols = ['Title', 'Posted Date', 'Location', 'Price', 'Description', 'Address', 'Beds', 'Baths', 'House Size', 'Land Size']

In [46]:
df_new_cols = pd.concat([df_merged['0'],df_merged['1'], df_merged['2'], df_merged['3'], df_merged['4'], df_merged['5'], df_merged['6'], df_merged['7'], 
                        df_merged['8'], df_merged['9']], axis=1)

In [48]:
df_new_cols.columns = cols

In [49]:
df_new_cols

,Title,Posted Date,Location,Price,Description,Address,Beds,Baths,House Size,Land Size
0,New House for Sale in Piliyandala,10 Aug 7:49 pm,"Piliyandala, Colombo",12500000,පිලියන්දල මඩපාත devid pieris ආයතනය අසල ඉදිකරමි...,NaN,4,2,"1,750.0 sqft",6.5 perches
1,Brand New Luxury Two Storey House Piliyanadala,2 Jul 8:33 am,"Kottawa, Colombo",29500000,Brand new luxury two storey house for sale in ...,kottawa 255 piliyanadala Road,4,4,"3,510.0 sqft",9.0 perches
2,Brand New 3 Storied House for Sale in Malabe,16 Aug 8:14 pm,"Malabe, Colombo",19000000,Brand New 3 storied house for sale at malabe ....,malabe,4,3,"2,850.0 sqft",7.5 perches
3,House for sale - Kottawa,16 Aug 8:32 pm,"Kottawa, Colombo",35000000,House for sale in Kottawa===================,NaN,7,4,"1,700.0 sqft",15.0 perches
4,Brand New Luxury House for sale in Boralesgamuwa,8 Aug 7:51 pm,"Boralesgamuwa, Colombo",29500000,"Brand New, 3 Storied 5 Bed Room House for Sale...",NaN,5,4,"3,500.0 sqft",8.0 perches
...,...,...,...,...,...,...,...,...,...,...
7607,House with Land for Sale in Nawala,1 Feb 7:26 am,"Nawala, Colombo",27000000,-Land with a house for sale-Only land value co...,"De Mel Watta Road, Koswatta",3,1,"2,500.0 sqft",9.0 perches
7608,Two Story House for Sale - Kohuwala,1 Feb 8:01 pm,"Kohuwala, Colombo",115000000,Two storey house for sale 5 bedrooms 5 bathroo...,s.de.s jayasinghe road kohuwala,5,5,"4,000.0 sqft",15.5 perches
7609,Brand New Luxury House at Kottawa Rd. Piliyandala,24 Jan 7:29 am,"Piliyandala, Colombo",13700000,Brand new luxury house at kottawa rd. piliyand...,kottawa rd. piliyandala,3,3,"1,456.0 sqft",10.0 perches
7610,2 Storey Facing Paddy Fields House for Sale Ba...,2 Feb 10:30 am,"Battaramulla, Colombo",35000000,2 Storey House Sale is overlooking vast paddy ...,Thalangama North Battharamulla,4,4,"4,200.0 sqft",12.0 perches


In [50]:
df_new_cols.to_csv( r"./" + 'house-data-set-right-header' + '.csv')

In [53]:
df_new_cols['House Size'] = df_new_cols['House Size'].apply(lambda x: str(x).replace('sqft', ''))

In [54]:
df_new_cols['Land Size'] = df_new_cols['Land Size'].apply(lambda x: str(x).replace('perches', ''))

In [55]:
df_new_cols.head()

,Title,Posted Date,Location,Price,Description,Address,Beds,Baths,House Size,Land Size
0,New House for Sale in Piliyandala,10 Aug 7:49 pm,"Piliyandala, Colombo",12500000,පිලියන්දල මඩපාත devid pieris ආයතනය අසල ඉදිකරමි...,NaN,4,2,"1,750.0",6.5
1,Brand New Luxury Two Storey House Piliyanadala,2 Jul 8:33 am,"Kottawa, Colombo",29500000,Brand new luxury two storey house for sale in ...,kottawa 255 piliyanadala Road,4,4,"3,510.0",9.0
2,Brand New 3 Storied House for Sale in Malabe,16 Aug 8:14 pm,"Malabe, Colombo",19000000,Brand New 3 storied house for sale at malabe ....,malabe,4,3,"2,850.0",7.5
3,House for sale - Kottawa,16 Aug 8:32 pm,"Kottawa, Colombo",35000000,House for sale in Kottawa===================,NaN,7,4,"1,700.0",15.0
4,Brand New Luxury House for sale in Boralesgamuwa,8 Aug 7:51 pm,"Boralesgamuwa, Colombo",29500000,"Brand New, 3 Storied 5 Bed Room House for Sale...",NaN,5,4,"3,500.0",8.0


In [56]:
df_new_cols.isnull().sum()

Title            3
Posted Date      3
Location         3
Price            0
Description      3
Address        944
Beds             0
Baths            0
House Size       0
Land Size        0
dtype: int64

In [57]:
df_new_cols['Location'] = df_new_cols['Location'].apply(lambda x: str(x).replace(', Colombo', ''))

In [58]:
df_new_cols

,Title,Posted Date,Location,Price,Description,Address,Beds,Baths,House Size,Land Size
0,New House for Sale in Piliyandala,10 Aug 7:49 pm,Piliyandala,12500000,පිලියන්දල මඩපාත devid pieris ආයතනය අසල ඉදිකරමි...,NaN,4,2,"1,750.0",6.5
1,Brand New Luxury Two Storey House Piliyanadala,2 Jul 8:33 am,Kottawa,29500000,Brand new luxury two storey house for sale in ...,kottawa 255 piliyanadala Road,4,4,"3,510.0",9.0
2,Brand New 3 Storied House for Sale in Malabe,16 Aug 8:14 pm,Malabe,19000000,Brand New 3 storied house for sale at malabe ....,malabe,4,3,"2,850.0",7.5
3,House for sale - Kottawa,16 Aug 8:32 pm,Kottawa,35000000,House for sale in Kottawa===================,NaN,7,4,"1,700.0",15.0
4,Brand New Luxury House for sale in Boralesgamuwa,8 Aug 7:51 pm,Boralesgamuwa,29500000,"Brand New, 3 Storied 5 Bed Room House for Sale...",NaN,5,4,"3,500.0",8.0
...,...,...,...,...,...,...,...,...,...,...
7607,House with Land for Sale in Nawala,1 Feb 7:26 am,Nawala,27000000,-Land with a house for sale-Only land value co...,"De Mel Watta Road, Koswatta",3,1,"2,500.0",9.0
7608,Two Story House for Sale - Kohuwala,1 Feb 8:01 pm,Kohuwala,115000000,Two storey house for sale 5 bedrooms 5 bathroo...,s.de.s jayasinghe road kohuwala,5,5,"4,000.0",15.5
7609,Brand New Luxury House at Kottawa Rd. Piliyandala,24 Jan 7:29 am,Piliyandala,13700000,Brand new luxury house at kottawa rd. piliyand...,kottawa rd. piliyandala,3,3,"1,456.0",10.0
7610,2 Storey Facing Paddy Fields House for Sale Ba...,2 Feb 10:30 am,Battaramulla,35000000,2 Storey House Sale is overlooking vast paddy ...,Thalangama North Battharamulla,4,4,"4,200.0",12.0


In [60]:
df_new_cols.isnull().sum()

Title            3
Posted Date      3
Location         0
Price            0
Description      3
Address        944
Beds             0
Baths            0
House Size       0
Land Size        0
dtype: int64

In [102]:
df_visible_features = pd.concat([df_new_cols['Location'], df_new_cols['Price'], df_new_cols['Beds'], df_new_cols['Baths'],
                                df_new_cols['House Size'], df_new_cols['Land Size']], axis =1)

In [103]:
df_visible_features.head()

,Location,Price,Beds,Baths,House Size,Land Size
0,Piliyandala,12500000,4,2,"1,750.0",6.5
1,Kottawa,29500000,4,4,"3,510.0",9.0
2,Malabe,19000000,4,3,"2,850.0",7.5
3,Kottawa,35000000,7,4,"1,700.0",15.0
4,Boralesgamuwa,29500000,5,4,"3,500.0",8.0


In [104]:
type(df_visible_features.head(1)['Land Size'][0])

str

In [105]:
df_visible_features['House Size'] = df_visible_features['House Size'].apply(lambda x:  float(str(x).replace(',', '')))

In [106]:
df_visible_features.head()

,Location,Price,Beds,Baths,House Size,Land Size
0,Piliyandala,12500000,4,2,1750.0,6.5
1,Kottawa,29500000,4,4,3510.0,9.0
2,Malabe,19000000,4,3,2850.0,7.5
3,Kottawa,35000000,7,4,1700.0,15.0
4,Boralesgamuwa,29500000,5,4,3500.0,8.0


In [107]:
df_visible_features['Land Size'] = df_visible_features['Land Size'].apply(lambda x:  {float(str(x).replace(',', '')))

SyntaxError: invalid syntax (<ipython-input-107-7a1bd216c19d>, line 1)

In [49]:
def landsize_converter(x):
    is_acres = False
#     x.replace(',', '')
    if 'acres' in x:
        x = x.replace('acres','')
        is_acres = True
    if ',' in x:
       x=  x.replace(',', '')
    if is_acres:
        x = 160 * float(x)
        return x
    else:
        x = float(x)
        return x

In [126]:
df_visible_features['Land Size'] = df_visible_features['Land Size'].apply(lambda x:  landsize_converter(str(x)))

In [127]:
df_visible_features.describe()

,Price,Beds,Baths,House Size,Land Size
count,7.609000e+03,7609.000000,7609.000000,7609.000000,7609.000000
mean,3.109433e+07,3.972795,3.141937,2778.529263,33.444613
std,4.069530e+07,1.186553,1.232166,2668.220947,1835.793791
min,8.500000e+04,1.000000,1.000000,174.240000,1.000000
25%,1.500000e+07,3.000000,2.000000,1750.000000,7.500000
50%,2.200000e+07,4.000000,3.000000,2600.000000,9.600000
75%,3.200000e+07,5.000000,4.000000,3250.000000,11.600000
max,1.300000e+09,10.000000,10.000000,140000.000000,160000.000000


In [128]:
df_visible_features.head()

,Location,Price,Beds,Baths,House Size,Land Size
0,Piliyandala,12500000,4,2,1750.0,6.5
1,Kottawa,29500000,4,4,3510.0,9.0
2,Malabe,19000000,4,3,2850.0,7.5
3,Kottawa,35000000,7,4,1700.0,15.0
4,Boralesgamuwa,29500000,5,4,3500.0,8.0


In [129]:
df_visible_features = df_visible_features.drop((df_visible_features[df_visible_features['Price'] == 0]).index, axis=0)

In [130]:
df_visible_features.describe()

,Price,Beds,Baths,House Size,Land Size
count,7.609000e+03,7609.000000,7609.000000,7609.000000,7609.000000
mean,3.109433e+07,3.972795,3.141937,2778.529263,33.444613
std,4.069530e+07,1.186553,1.232166,2668.220947,1835.793791
min,8.500000e+04,1.000000,1.000000,174.240000,1.000000
25%,1.500000e+07,3.000000,2.000000,1750.000000,7.500000
50%,2.200000e+07,4.000000,3.000000,2600.000000,9.600000
75%,3.200000e+07,5.000000,4.000000,3250.000000,11.600000
max,1.300000e+09,10.000000,10.000000,140000.000000,160000.000000


In [131]:
df_visible_features.to_csv(r'./house_data_set_cleaned_2.csv')

In [132]:
df_check = pd.read_csv('./house_data_set_cleaned_2.csv')

In [133]:
df_check.describe()

,Unnamed: 0,Price,Beds,Baths,House Size,Land Size
count,7609.000000,7.609000e+03,7609.000000,7609.000000,7609.000000,7609.000000
mean,3805.459193,3.109433e+07,3.972795,3.141937,2778.529263,33.444613
std,2197.852576,4.069530e+07,1.186553,1.232166,2668.220947,1835.793791
min,0.000000,8.500000e+04,1.000000,1.000000,174.240000,1.000000
25%,1902.000000,1.500000e+07,3.000000,2.000000,1750.000000,7.500000
50%,3805.000000,2.200000e+07,4.000000,3.000000,2600.000000,9.600000
75%,5709.000000,3.200000e+07,5.000000,4.000000,3250.000000,11.600000
max,7611.000000,1.300000e+09,10.000000,10.000000,140000.000000,160000.000000


In [134]:
df_check.head()

,Unnamed: 0,Location,Price,Beds,Baths,House Size,Land Size
0,0,Piliyandala,12500000,4,2,1750.0,6.5
1,1,Kottawa,29500000,4,4,3510.0,9.0
2,2,Malabe,19000000,4,3,2850.0,7.5
3,3,Kottawa,35000000,7,4,1700.0,15.0
4,4,Boralesgamuwa,29500000,5,4,3500.0,8.0


In [20]:
df_old = pd.read_csv('house-data-set-right-header' + '.csv')

In [21]:
df_old

,Unnamed: 0,Title,Posted Date,Location,Price,Description,Address,Beds,Baths,House Size,Land Size
0,0,New House for Sale in Piliyandala,10 Aug 7:49 pm,"Piliyandala, Colombo",12500000,පිලියන්දල මඩපාත devid pieris ආයතනය අසල ඉදිකරමි...,NaN,4,2,"1,750.0 sqft",6.5 perches
1,1,Brand New Luxury Two Storey House Piliyanadala,2 Jul 8:33 am,"Kottawa, Colombo",29500000,Brand new luxury two storey house for sale in ...,kottawa 255 piliyanadala Road,4,4,"3,510.0 sqft",9.0 perches
2,2,Brand New 3 Storied House for Sale in Malabe,16 Aug 8:14 pm,"Malabe, Colombo",19000000,Brand New 3 storied house for sale at malabe ....,malabe,4,3,"2,850.0 sqft",7.5 perches
3,3,House for sale - Kottawa,16 Aug 8:32 pm,"Kottawa, Colombo",35000000,House for sale in Kottawa===================,NaN,7,4,"1,700.0 sqft",15.0 perches
4,4,Brand New Luxury House for sale in Boralesgamuwa,8 Aug 7:51 pm,"Boralesgamuwa, Colombo",29500000,"Brand New, 3 Storied 5 Bed Room House for Sale...",NaN,5,4,"3,500.0 sqft",8.0 perches
...,...,...,...,...,...,...,...,...,...,...,...
7607,7607,House with Land for Sale in Nawala,1 Feb 7:26 am,"Nawala, Colombo",27000000,-Land with a house for sale-Only land value co...,"De Mel Watta Road, Koswatta",3,1,"2,500.0 sqft",9.0 perches
7608,7608,Two Story House for Sale - Kohuwala,1 Feb 8:01 pm,"Kohuwala, Colombo",115000000,Two storey house for sale 5 bedrooms 5 bathroo...,s.de.s jayasinghe road kohuwala,5,5,"4,000.0 sqft",15.5 perches
7609,7609,Brand New Luxury House at Kottawa Rd. Piliyandala,24 Jan 7:29 am,"Piliyandala, Colombo",13700000,Brand new luxury house at kottawa rd. piliyand...,kottawa rd. piliyandala,3,3,"1,456.0 sqft",10.0 perches
7610,7610,2 Storey Facing Paddy Fields House for Sale Ba...,2 Feb 10:30 am,"Battaramulla, Colombo",35000000,2 Storey House Sale is overlooking vast paddy ...,Thalangama North Battharamulla,4,4,"4,200.0 sqft",12.0 perches


In [22]:
new_houses = []
with open('df', 'rb') as file:
        new_houses = pickle.load(file)

In [24]:
df_new = pd.DataFrame(new_houses)

In [27]:
df_new.columns = cols = ['Title', 'Posted Date', 'Location', 'Price', 'Description', 'Address', 'Beds', 'Baths', 'House Size', 'Land Size']

In [28]:
df_new

,Title,Posted Date,Location,Price,Description,Address,Beds,Baths,House Size,Land Size
0,,,,0,,,0,0,0.0 sqft,0.0 perches
1,Luxury House for sale in Boralesgamuwa,1 Mar 1:06 am,"Boralesgamuwa, Colombo","29,000,000",* Two story 4 bedroom house for sale in Borale...,,4,4,"2,500.0 sqft",11.0 perches
2,Brand New Two Story House for Sale in Kottawa,28 Mar 12:04 pm,"Kottawa, Colombo","25,000,000",Brand New Two Story House in KottawaIdeal for ...,kottawa,4,4,"3,200.0 sqft",7.5 perches
3,House For Sale Kaduwela,6 Mar 7:44 am,"Kaduwela, Colombo","9,800,000",Fully Completed Single Story house for sale i...,"Kalaniya Road, Mabima, Kaduwela",3,2,"1,200.0 sqft",14.0 perches
4,Luxury House for Sale in Piliyandala,6 Feb 6:42 am,"Piliyandala, Colombo","17,000,000",brand new 2 Story modern luxury ARCHITECTUR...,KESBEWA CITY,4,4,"1,950.0 sqft",6.2 perches
...,...,...,...,...,...,...,...,...,...,...
5095,"House for Sale in Baddagana, Pita Kotte",25 Feb 6:10 pm,"Kotte, Colombo","55,000,000","Two storey house for sale in pita kotte, It is...",,7,4,"3,500.0 sqft",15.2 perches
5096,Brand New Mordern House for Sale in Koralaima,15 Mar 6:58 am,"Piliyandala, Colombo","7,800,000","# තවත් සති 2 න් වැඩ නිම කරන, ඉහලම ප්‍රමිතියෙන්...","Koralaima, Kahatuduwa",3,2,"1,400.0 sqft",7.5 perches
5097,2 Storied House for Sale in Malabe,14 Mar 5:53 pm,"Malabe, Colombo","17,000,000",*- 700 M to 190 Bus Route*- 2.5 km Athurugiriy...,"Bakmeegaha Rd , Malabe.",4,4,"2,000.0 sqft",7.5 perches
5098,New Luxury Ultra Modern Two Story House Malabe,23 Feb 12:53 am,"Malabe, Colombo","27,500,000",# පර්චස් 22 ක ඉඩමේ ඉදි කොට ඇති නවීන පන්නයේ අංග...,"Vihara Mawatha, Malabe",5,4,"2,200.0 sqft",22.32 perches


In [39]:
type(pd.concat([df_old.drop(['Unnamed: 0'], axis=1), df_new]).drop(['Title', 'Posted Date', 'Description', 'Address'], axis=1))

pandas.core.frame.DataFrame

In [38]:
pd.concat([df_old.drop(['Unnamed: 0'], axis=1), df_new]).drop(['Title', 'Posted Date', 'Description', 'Address'], axis=1).to_csv('house_data_set_apr.csv')

In [40]:
df_saved = pd.read_csv('./house_data_set_apr.csv')

In [41]:
df_saved

,Unnamed: 0,Location,Price,Beds,Baths,House Size,Land Size
0,0,"Piliyandala, Colombo",12500000,4,2,"1,750.0 sqft",6.5 perches
1,1,"Kottawa, Colombo",29500000,4,4,"3,510.0 sqft",9.0 perches
2,2,"Malabe, Colombo",19000000,4,3,"2,850.0 sqft",7.5 perches
3,3,"Kottawa, Colombo",35000000,7,4,"1,700.0 sqft",15.0 perches
4,4,"Boralesgamuwa, Colombo",29500000,5,4,"3,500.0 sqft",8.0 perches
...,...,...,...,...,...,...,...
12707,5095,"Kotte, Colombo","55,000,000",7,4,"3,500.0 sqft",15.2 perches
12708,5096,"Piliyandala, Colombo","7,800,000",3,2,"1,400.0 sqft",7.5 perches
12709,5097,"Malabe, Colombo","17,000,000",4,4,"2,000.0 sqft",7.5 perches
12710,5098,"Malabe, Colombo","27,500,000",5,4,"2,200.0 sqft",22.32 perches


In [44]:
df_saved['Location'] = df_saved['Location'].apply(lambda x: str(x).replace(', Colombo', ''))

In [45]:
df_saved.head()

,Unnamed: 0,Location,Price,Beds,Baths,House Size,Land Size
0,0,Piliyandala,12500000,4,2,"1,750.0 sqft",6.5 perches
1,1,Kottawa,29500000,4,4,"3,510.0 sqft",9.0 perches
2,2,Malabe,19000000,4,3,"2,850.0 sqft",7.5 perches
3,3,Kottawa,35000000,7,4,"1,700.0 sqft",15.0 perches
4,4,Boralesgamuwa,29500000,5,4,"3,500.0 sqft",8.0 perches


In [47]:
df_saved['House Size'] = df_saved['House Size'].apply(lambda x: str(x).replace('sqft', ''))

In [48]:
df_saved['Land Size'] = df_saved['Land Size'].apply(lambda x: str(x).replace('perches', ''))

In [51]:
df_saved['Land Size'] = df_saved['Land Size'].apply(lambda x:  landsize_converter(str(x)))

In [55]:
df_saved.to_csv('house_dataset_apr_processed.csv')

In [58]:
df_saved['House Size'] = df_saved['House Size'].apply(lambda x:  float(str(x).replace(',', '')))

In [61]:
df_saved['Price'] = df_saved['Price'].apply(lambda x:  float(str(x).replace(',', '')))

In [65]:
df_saved['Beds'] = df_saved['Beds'].apply(lambda x:  float(str(x).replace('+', '')))

In [66]:
df_saved['Baths'] = df_saved['Baths'].apply(lambda x:  float(str(x).replace('+', '')))

In [67]:
df_saved.describe()

,Unnamed: 0,Price,Beds,Baths,House Size,Land Size
count,12712.000000,1.271200e+04,12712.000000,12712.000000,12712.000000,12712.000000
mean,3301.598175,2.790595e+07,3.595107,2.859739,2503.372860,23.491028
std,2034.761556,3.708087e+07,1.616019,1.476507,2369.199635,1420.541178
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,1588.750000,1.200000e+07,3.000000,2.000000,1500.000000,7.000000
50%,3177.500000,2.100000e+07,4.000000,3.000000,2500.000000,8.800000
75%,4766.250000,3.000000e+07,4.000000,4.000000,3200.000000,11.000000
max,7611.000000,1.300000e+09,10.000000,10.000000,140000.000000,160000.000000


In [68]:
df_saved.to_csv('house_dataset_apr_processed.csv')

In [69]:
df_saved.isnull().sum()

Unnamed: 0    0
Location      0
Price         0
Beds          0
Baths         0
House Size    0
Land Size     0
dtype: int64

In [72]:
df_saved[df_saved['Land Size'] < 30].shape

(12447, 7)